In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score

In [2]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [3]:
def refine_data(arr1,arr2,arr3):
    refined_data = []
    for i in range(arr1.shape[0]):
        rt = find_nearest(arr1[i],arr3[i][0])
        refined_data.append([rt,arr2[i][arr1[i].tolist().index(rt)]])
        
    refined_data = np.array(refined_data)
    return refined_data

In [4]:
def get_metrics(arr1,arr2):
    mse = mean_squared_error(arr1,arr2)
    r2 = r2_score(arr1,arr2)
    return mse,r2

In [5]:
d1 = pd.read_csv('X_data_t1',sep = ',')
d2 = pd.read_csv('y_data',sep = ',')
d3 = pd.concat([d1,d2],axis=1)

#data = d3.loc[(d3['maxRT_ab'] < 1e6) & (d3['maxRT_ab'] > 3e4)]
#data = d3.loc[(d3['maxRT_ab'] > 3e4)]
#data = d3.loc[(d3['maxRT_ab'] < 1e6)]
#d3 = shuffle(d3)

#X = np.array(d3[['maxRT_t','x_start_t','diff_start','diff_end']])
#X = np.array(d3[['maxRT_ab','maxRT_baseline','x_start_ab','x_end_ab']])
#X = np.array(d3[['maxRT_t','maxRT_ab','maxRT_baseline','x_start_ab','x_end_ab']])
X = np.array(d3[['maxRT_t','maxRT_ab','maxRT_baseline','x_start_t','x_start_ab','diff_start','x_end_t','x_end_ab','diff_end']])

y = np.array(d3[['y_right_t','y_right_ab']])


time = np.array(pd.read_csv('time',sep = ',',header = None).dropna(axis = 'columns').round(3))
abundance = np.array(pd.read_csv('abundance',sep = ',',header = None).dropna(axis = 'columns'))
baseline = np.array(pd.read_csv('baseline',sep = ',',header = None).dropna(axis = 'columns'))
#time = time[0:420]
#abundance = abundance[0:420]
#baseline = baseline[0:420]

In [6]:
np.amax(d3['maxRT_ab']), np.amin(d3['maxRT_ab']), X.shape

(99672936.0, 22512.0, (423, 9))

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
#scaler_train = MinMaxScaler()
scaler_train = StandardScaler()
scaler_train.fit(X_train)
X_train =scaler_train.transform(X_train)

#scaler_test = MinMaxScaler()
scaler_test = StandardScaler()
scaler_test.fit(X_test)
X_test = scaler_test.transform(X_test)

In [9]:
n_estimators = 1000
max_depth = 1000
random_state = 1000

#mse_t,r2_t = [[]for i in range(len(n_estimators))], [[] for i in range(len(n_estimators))]
#mse_ab,r2_ab = [[]for i in range(len(n_estimators))], [[] for i in range(len(n_estimators))]


params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': 20,
                 'n_jobs': 2, 'random_state':random_state}
        
clf = RandomForestRegressor(**params)
scores = cross_val_score(clf, X_train, y_train, cv=5)
print(scores)
clf.fit(X_train,y_train)
y_predict = clf.predict(X_test).round(3)
        
        
mse_1,r2_1 = get_metrics(y_test[:,0],y_predict[:,0])
mse_2,r2_2 = get_metrics(y_test[:,1],y_predict[:,1])
        
        
print(r2_1,r2_2)
print(mse_1,mse_2)

        

[0.80453945 0.63004094 0.42585301 0.62889953 0.75275755]
0.2402010655519894 0.7927142548322196
0.017507223529411763 1630966.3274087885


In [10]:
y_test[:,0]

array([1.717, 1.367, 1.223, 1.36 , 1.757, 1.46 , 1.707, 1.37 , 1.507,
       1.713, 1.707, 1.243, 1.69 , 1.44 , 1.66 , 1.703, 1.457, 1.68 ,
       1.703, 1.503, 1.447, 1.723, 1.3  , 1.557, 1.5  , 1.443, 1.7  ,
       1.663, 1.72 , 1.707, 1.697, 1.38 , 1.737, 1.703, 1.437, 1.293,
       1.703, 1.557, 1.737, 1.397, 1.7  , 1.7  , 1.307, 1.45 , 1.507,
       1.64 , 1.477, 1.383, 1.64 , 1.577, 1.677, 1.553, 1.7  , 1.503,
       1.737, 1.61 , 1.7  , 1.433, 1.713, 1.703, 1.753, 1.717, 1.7  ,
       1.747, 1.397, 1.657, 1.5  , 1.703, 1.307, 1.7  , 1.45 , 1.4  ,
       1.473, 1.273, 1.44 , 1.513, 1.697, 1.69 , 1.72 , 1.53 , 1.72 ,
       1.707, 1.323, 1.587, 1.41 ])

In [11]:
y_predict[:,0]

array([1.706, 1.499, 1.491, 1.556, 1.502, 1.545, 1.702, 1.503, 1.618,
       1.625, 1.7  , 1.609, 1.639, 1.454, 1.605, 1.556, 1.504, 1.657,
       1.699, 1.516, 1.51 , 1.665, 1.556, 1.505, 1.54 , 1.539, 1.559,
       1.592, 1.703, 1.703, 1.525, 1.533, 1.633, 1.7  , 1.501, 1.496,
       1.695, 1.578, 1.541, 1.499, 1.705, 1.689, 1.538, 1.523, 1.488,
       1.492, 1.637, 1.503, 1.609, 1.536, 1.623, 1.531, 1.669, 1.514,
       1.493, 1.543, 1.534, 1.556, 1.639, 1.698, 1.556, 1.612, 1.705,
       1.549, 1.534, 1.612, 1.597, 1.683, 1.494, 1.694, 1.529, 1.6  ,
       1.577, 1.6  , 1.505, 1.51 , 1.543, 1.548, 1.556, 1.556, 1.494,
       1.533, 1.615, 1.498, 1.491])

In [12]:
from sklearn.decomposition.pca import PCA
from sklearn.preprocessing import scale
pca = PCA()
X_reduced = pca.fit_transform(X_train)
scores = cross_val_score(clf, X_reduced, y_train, cv=5, scoring = 'neg_mean_squared_error')
print(scores)
clf.fit(X_reduced,y_train)
y_predict = clf.predict(X_test).round(3)
        
        
mse_1,r2_1 = get_metrics(y_test[:,0],y_predict[:,0])
mse_2,r2_2 = get_metrics(y_test[:,1],y_predict[:,1])
        
        
print(r2_1,r2_2)
print(mse_1,mse_2)

[ -563624.72802712  -311455.61769407 -2265695.16701772 -2612900.0101347
 -1495903.31423237]
-0.506646536020174 -0.9378385684682478
0.034716023529411756 15247307.28862982


In [13]:
y_predict[:,0]

array([1.54 , 1.649, 1.645, 1.65 , 1.659, 1.649, 1.534, 1.647, 1.647,
       1.629, 1.557, 1.646, 1.531, 1.668, 1.645, 1.645, 1.648, 1.532,
       1.554, 1.649, 1.648, 1.596, 1.649, 1.65 , 1.659, 1.65 , 1.653,
       1.65 , 1.533, 1.546, 1.648, 1.645, 1.646, 1.559, 1.653, 1.65 ,
       1.547, 1.647, 1.643, 1.648, 1.544, 1.554, 1.65 , 1.658, 1.649,
       1.648, 1.552, 1.66 , 1.648, 1.647, 1.65 , 1.649, 1.578, 1.646,
       1.649, 1.61 , 1.648, 1.668, 1.644, 1.544, 1.649, 1.619, 1.54 ,
       1.648, 1.701, 1.648, 1.649, 1.545, 1.645, 1.532, 1.649, 1.659,
       1.65 , 1.649, 1.647, 1.648, 1.649, 1.647, 1.649, 1.65 , 1.65 ,
       1.649, 1.643, 1.649, 1.658])

In [14]:
y_test[:,0]

array([1.717, 1.367, 1.223, 1.36 , 1.757, 1.46 , 1.707, 1.37 , 1.507,
       1.713, 1.707, 1.243, 1.69 , 1.44 , 1.66 , 1.703, 1.457, 1.68 ,
       1.703, 1.503, 1.447, 1.723, 1.3  , 1.557, 1.5  , 1.443, 1.7  ,
       1.663, 1.72 , 1.707, 1.697, 1.38 , 1.737, 1.703, 1.437, 1.293,
       1.703, 1.557, 1.737, 1.397, 1.7  , 1.7  , 1.307, 1.45 , 1.507,
       1.64 , 1.477, 1.383, 1.64 , 1.577, 1.677, 1.553, 1.7  , 1.503,
       1.737, 1.61 , 1.7  , 1.433, 1.713, 1.703, 1.753, 1.717, 1.7  ,
       1.747, 1.397, 1.657, 1.5  , 1.703, 1.307, 1.7  , 1.45 , 1.4  ,
       1.473, 1.273, 1.44 , 1.513, 1.697, 1.69 , 1.72 , 1.53 , 1.72 ,
       1.707, 1.323, 1.587, 1.41 ])

In [15]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

array([ 73.62,  88.95,  94.53,  98.66,  99.8 ,  99.99, 100.  , 100.  ,
       100.  ])